### Reading training data and pruning/compaction

In [2]:
# load some default Python modules
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import feather
% matplotlib inline
plt.style.use('seaborn-whitegrid')

In [2]:
# Set columns to most suitable type to optimize for memory usage
traintypes = {'fare_amount': 'float32',
              'pickup_datetime': 'str',
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

cols = list(traintypes.keys())

In [8]:
to_datetime = lambda d: datetime.strptime(d, '%Y-%m-%d %H:%M:%S UTC')

In [4]:
print(datetime.now())
df_train =  pd.read_csv('../../datasets/kaggle/new-york-city-taxi-fare-prediction/train.csv',converters={'pickup_datetime': to_datetime},usecols=cols, dtype=traintypes)
print(datetime.now())

2018-09-18 21:39:45.822030


/home/subhachandra/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Both a converter and dtype were specified for column pickup_datetime - only the converter will be used
  from ipykernel import kernelapp as app


2018-09-18 21:46:42.554108


Converting float64 to float32 and passenger_count to uint8 reduced the dataframe size  from around 3.95GB to 1.5GB

In [5]:
print(df_train.info())
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55423856 entries, 0 to 55423855
Data columns (total 7 columns):
fare_amount          float32
pickup_datetime      datetime64[ns]
pickup_longitude     float32
pickup_latitude      float32
dropoff_longitude    float32
dropoff_latitude     float32
passenger_count      uint8
dtypes: datetime64[ns](1), float32(5), uint8(1)
memory usage: 1.5 GB
None


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21,-73.844315,40.721317,-73.841614,40.712276,1
1,16.9,2010-01-05 16:52:16,-74.016045,40.711304,-73.979271,40.782005,1
2,5.7,2011-08-18 00:35:00,-73.982735,40.761269,-73.991241,40.750561,2
3,7.7,2012-04-21 04:30:42,-73.987129,40.733143,-73.991570,40.758091,1
4,5.3,2010-03-09 07:51:00,-73.968094,40.768009,-73.956657,40.783764,1


Drop rows that have -ve fare values or any null values.

In [6]:
print('Old size: %d' % len(df_train))
df_train = df_train[df_train.fare_amount>=0]
print('Size after dropping -ve fares: %d' % len(df_train))
df_train = df_train.dropna(how = 'any', axis = 'rows')
print('Size after dropping rows with missing data: %d' % len(df_train))

Old size: 55423856
Size after dropping -ve fares: 55421402
Size after dropping rows with missing data: 55421026


In [7]:
print(datetime.now())
df_train.describe()
print(datetime.now())

2018-09-18 21:46:50.117474
2018-09-18 21:47:04.459308


In [9]:
# Set columns to most suitable type to optimize for memory usage
testtypes = { 'key': 'str',
              'pickup_datetime': 'str',
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

testcols = list(testtypes.keys())
df_test =  pd.read_csv('../../datasets/kaggle/new-york-city-taxi-fare-prediction/test.csv',converters={'pickup_datetime': to_datetime},usecols=testcols, dtype=testtypes)
df_test.describe()

/home/subhachandra/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:11: ParserWarning: Both a converter and dtype were specified for column pickup_datetime - only the converter will be used


,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974716,40.751041,-73.973656,40.751740,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252190,40.573143,-74.263245,40.568974,1.000000
25%,-73.992500,40.736125,-73.991249,40.735253,1.000000
50%,-73.982327,40.753052,-73.980015,40.754065,1.000000
75%,-73.968012,40.767113,-73.964062,40.768757,2.000000
max,-72.986534,41.709557,-72.990967,41.696682,6.000000


In [10]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44,-73.982521,40.751259,-73.979652,40.746140,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12,-73.981163,40.767807,-73.990448,40.751637,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12,-73.966049,40.789776,-73.988564,40.744427,1


Look at the __Location Data__ and the sections after that from Albert Van Bremen's notebook for visualizations of the dataset on a nyc map and how to prune outliers and bad data.

In [10]:
# minimum and maximum longitude test set
print("min,max longitude",min(df_test.pickup_longitude.min(), df_test.dropoff_longitude.min()),
max(df_test.pickup_longitude.max(), df_test.dropoff_longitude.max()))

# minimum and maximum latitude test
print("min,max latitude",min(df_test.pickup_latitude.min(), df_test.pickup_latitude.min()),
max(df_test.pickup_latitude.max(), df_test.pickup_latitude.max()))

min,max longitude -74.263245 -72.986534
min,max latitude 40.573143 41.709557


Prune from training set trips that are outside the bounds of the test set.

In [11]:
# This function will also be used with the test set below
def select_within_boundingbox(df, BB):
    return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
           (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
           (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
           (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])
            
# load image of NYC map
BB = (-74.5, -72.8, 40.5, 41.8)
#nyc_map = plt.imread('https://aiblog.nl/download/nyc_-74.5_-72.8_40.5_41.8.png')

# load extra image to zoom in on NYC
BB_zoom = (-74.3, -73.7, 40.5, 40.9)
#nyc_map_zoom = plt.imread('https://aiblog.nl/download/nyc_-74.3_-73.7_40.5_40.9.png')

In [12]:
print('Old size: %d' % len(df_train))
df_train = df_train[select_within_boundingbox(df_train, BB)]
print('Size after bounding based on test data: %d' % len(df_train))

Old size: 55421026
Size after bounding based on test data: 54246832


Compute distance in miles from the longitude/latitude information. Note that this is not the actual distance travelled on the streets but a straight line distance between the start and end.

In [13]:
# For this plot and further analysis, we need a function to calculate the distance in miles between locations in lon,lat coordinates.
# This function is based on https://stackoverflow.com/questions/27928/
# calculate-distance-between-two-latitude-longitude-points-haversine-formula 
# return distance in miles
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...

In [14]:
print(datetime.now())
# Add new column to dataframe for distance of trip in miles
df_train['distance_miles'] = distance(df_train.pickup_latitude, df_train.pickup_longitude, \
                                      df_train.dropoff_latitude, df_train.dropoff_longitude)
print(datetime.now())

2018-09-18 21:47:06.005116
2018-09-18 21:47:07.956941


In [15]:
df_train.distance_miles.describe()

count    5.424683e+07
mean     1.859483e+00
std      2.476777e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.366828e+00
75%      2.424845e+00
max      8.938039e+01
Name: distance_miles, dtype: float64

In [16]:
jfk = (-73.7822222222, 40.6441666667)
nyc = (-73.96, 40.78)
ewr = (-74.175, 40.69) # Newark Liberty International Airport, see https://www.travelmath.com/airport/EWR
lgr = (-73.87, 40.77) # LaGuardia Airport, see https://www.travelmath.com/airport/LGA

nyc_airports={
    'JFK':{'min_lng':-73.8352,
     'min_lat':40.6195,
     'max_lng':-73.7401, 
     'max_lat':40.6659},
              
    'EWR':{'min_lng':-74.1925,
            'min_lat':40.6700, 
            'max_lng':-74.1531, 
            'max_lat':40.7081

        },
    'LGA':{'min_lng':-73.8895, 
                  'min_lat':40.7664, 
                  'max_lng':-73.8550, 
                  'max_lat':40.7931
        
    }
    
}
def isAirport(latitude,longitude,airport_name='JFK'):   
    return (latitude>=nyc_airports[airport_name]['min_lat'] 
            and latitude<=nyc_airports[airport_name]['max_lat'] 
            and longitude>=nyc_airports[airport_name]['min_lng'] 
            and longitude<=nyc_airports[airport_name]['max_lng'])


In [17]:
#df_small = df_train[0:100000].copy()

In [18]:
print(datetime.now())
#df_small['is_pickup_JFK']=df_small.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'JFK'),axis=1)
#df_small['is_pickup_JFK']= df_small.apply(isAirport,axis=1)
p_jfk, d_jfk, p_ewr, d_ewr, p_lga, d_lga, pd_jfk = [],[],[],[],[],[],[]
start_time = datetime.now()
for i, row in df_train.iterrows():
    p_j = isAirport(row['pickup_latitude'],row['pickup_longitude'],'JFK')
    p_jfk.append(p_j)
    d_j = isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'JFK')
    d_jfk.append(d_j)
    pd_jfk.append(p_j or d_j)
    p_e = isAirport(row['pickup_latitude'],row['pickup_longitude'],'EWR')
    p_ewr.append(p_e)
    d_e = isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'EWR')
    d_ewr.append(d_e)
    p_l = isAirport(row['pickup_latitude'],row['pickup_longitude'],'LGA')
    p_lga.append(p_l)
    d_l = isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'LGA')
    d_lga.append(d_l)
    if (i+1) % 1000000 == 0:
        print(i+1,datetime.now() - start_time)

df_train['is_pickup_JFK_new'] = p_jfk
df_train['is_dropoff_JFK_new'] = d_jfk
df_train['is_pickup_EWR_new'] = p_ewr
df_train['is_dropoff_EWR_new'] = d_ewr
df_train['is_pickup_LGA_new'] = p_lga
df_train['is_dropoff_LGA_new'] = d_lga
df_train['is_to_from_JFK_new'] = pd_jfk
print(datetime.now())


2018-09-18 21:47:09.690928
1000000 0:03:13.948306
2000000 0:04:49.281696
3000000 0:06:24.063885
4000000 0:07:58.445041
5000000 0:09:33.051861
6000000 0:11:07.305182
7000000 0:12:41.860940
8000000 0:14:16.118840
9000000 0:15:51.001033
10000000 0:17:26.526531
11000000 0:19:01.684722
12000000 0:20:36.016065
13000000 0:22:11.083678
14000000 0:23:46.222639
15000000 0:25:20.584113
16000000 0:26:55.570963
18000000 0:30:05.133833
19000000 0:31:39.251624
20000000 0:33:14.041228
21000000 0:34:49.105001
23000000 0:37:59.042193
24000000 0:39:34.092421
25000000 0:41:09.042665
26000000 0:42:44.265170
27000000 0:44:19.516856
28000000 0:45:54.023652
29000000 0:47:28.653041
30000000 0:49:03.384378
31000000 0:50:37.325162
32000000 0:52:12.537552
33000000 0:53:47.449401
34000000 0:55:21.802225
35000000 0:56:56.523617
36000000 0:58:30.667058
37000000 1:00:05.583874
39000000 1:03:15.778428
40000000 1:04:52.208671
41000000 1:06:28.048622
42000000 1:08:04.179524
43000000 1:09:40.280280
44000000 1:11:16.38157

In [19]:
df_train[900000:900010]


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance_miles,is_pickup_JFK_new,is_dropoff_JFK_new,is_pickup_EWR_new,is_dropoff_EWR_new,is_pickup_LGA_new,is_dropoff_LGA_new,is_to_from_JFK_new
919236,9.000000,2012-09-27 10:55:01,-73.990395,40.751610,-73.991135,40.765697,1,1.366828,False,False,False,False,False,False,False
919237,6.000000,2012-10-29 18:31:00,-73.999466,40.743984,-73.986015,40.762417,5,1.366828,False,False,False,False,False,False,False
919238,18.500000,2014-11-04 09:41:47,-73.951828,40.782024,-73.983475,40.762074,1,2.254467,False,False,False,False,False,False,False
919239,12.100000,2010-08-19 11:37:00,-74.002419,40.724072,-73.981483,40.741116,5,1.714922,False,False,False,False,False,False,False
919240,13.700000,2010-09-01 10:00:07,-73.984001,40.761482,-73.976372,40.758980,1,0.000000,False,False,False,False,False,False,False
919241,9.500000,2014-05-13 06:48:00,-73.968124,40.762463,-73.932388,40.751881,1,1.793277,False,False,False,False,False,False,False
919242,13.000000,2014-08-06 12:00:00,-74.006355,40.737534,-73.985939,40.722069,3,1.714949,False,False,False,False,False,False,False
919243,16.000000,2015-04-12 02:40:30,-73.989029,40.718910,-73.986908,40.765179,1,3.056320,False,False,False,False,False,False,False
919244,49.799999,2012-06-11 14:41:00,-73.780479,40.645264,-73.989075,40.758602,1,13.447493,True,False,False,False,False,False,True
919245,6.500000,2012-09-19 19:14:28,-73.985718,40.740921,-73.973305,40.754921,1,1.366828,False,False,False,False,False,False,False


Compute distance from city center to the start of the trip.

In [20]:
print(datetime.now())
# Add new column to dataframe for distance from city center to the starting point
df_train['distance_to_center'] = distance(nyc[1], nyc[0], df_train.pickup_latitude, df_train.pickup_longitude)
print(datetime.now())

2018-09-18 23:16:54.097655
2018-09-18 23:16:55.708953


In [21]:
df_train.head()
path_train = '../../datasets/kaggle/new-york-city-taxi-fare-prediction/train2.feather'
feather.write_dataframe(df_train, path_train)

In [22]:
import gc
del df_train
gc.collect()

21

In [3]:
df_train = feather.read_dataframe("../../datasets/kaggle/new-york-city-taxi-fare-prediction/train2.feather")

Split pickup time into additional features - hour, weekday and year.

In [4]:
# Add new columns to dataframe for hour of the day, day of the week and year
print(datetime.now())
df_train['hour'] = df_train.pickup_datetime.apply(lambda t: t.hour)
df_train['weekday'] = df_train.pickup_datetime.apply(lambda t: t.weekday())
df_train['year'] = df_train.pickup_datetime.apply(lambda t: t.year)
print(datetime.now())
df_train.info()

2018-09-19 07:51:58.377407
2018-09-19 07:57:21.193761
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54246832 entries, 0 to 54246831
Data columns (total 19 columns):
fare_amount           float32
pickup_datetime       datetime64[ns]
pickup_longitude      float32
pickup_latitude       float32
dropoff_longitude     float32
dropoff_latitude      float32
passenger_count       uint8
distance_miles        float32
is_pickup_JFK_new     bool
is_dropoff_JFK_new    bool
is_pickup_EWR_new     bool
is_dropoff_EWR_new    bool
is_pickup_LGA_new     bool
is_dropoff_LGA_new    bool
is_to_from_JFK_new    bool
distance_to_center    float32
hour                  int64
weekday               int64
year                  int64
dtypes: bool(7), datetime64[ns](1), float32(7), int64(3), uint8(1)
memory usage: 3.4 GB


In [5]:
BB = (-74.5, -72.8, 40.5, 41.8)
manhattan = (-74.0479,-73.9067,40.6829,40.8820)
queens = (-73.9630,-73.7004,40.5431,40.8007)
brooklyn = (-74.0421,-73.8334,40.5707,40.7395)
bronx = (-73.9339,-73.7654,40.7855,40.9176)
staten = (-74.2558,-74.0522,40.4960,40.6490)

def isBorough(latitude,longitude,borough_name='manhattan'):
    
    if (latitude>=nyc_boroughs[borough_name]['min_lat'] and 
        latitude<=nyc_boroughs[borough_name]['max_lat'] and 
        longitude>=nyc_boroughs[borough_name]['min_lng'] and 
        longitude<=nyc_boroughs[borough_name]['max_lng']):
        return 1
    else:
        return 0

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
key                  9914 non-null object
pickup_datetime      9914 non-null datetime64[ns]
pickup_longitude     9914 non-null float32
pickup_latitude      9914 non-null float32
dropoff_longitude    9914 non-null float32
dropoff_latitude     9914 non-null float32
passenger_count      9914 non-null uint8
dtypes: datetime64[ns](1), float32(4), object(1), uint8(1)
memory usage: 319.6+ KB


Add the new features to the test set too.

In [14]:
df_test['distance_miles'] = distance(df_test.pickup_latitude, df_test.pickup_longitude, \
                                      df_test.dropoff_latitude, df_test.dropoff_longitude)

In [17]:
print(datetime.now())
#df_small['is_pickup_JFK']=df_small.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'JFK'),axis=1)
#df_small['is_pickup_JFK']= df_small.apply(isAirport,axis=1)
p_jfk, d_jfk, p_ewr, d_ewr, p_lga, d_lga, pd_jfk = [],[],[],[],[],[],[]
start_time = datetime.now()
for i, row in df_test.iterrows():
    p_j = isAirport(row['pickup_latitude'],row['pickup_longitude'],'JFK')
    p_jfk.append(p_j)
    d_j = isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'JFK')
    d_jfk.append(d_j)
    pd_jfk.append(p_j or d_j)
    p_e = isAirport(row['pickup_latitude'],row['pickup_longitude'],'EWR')
    p_ewr.append(p_e)
    d_e = isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'EWR')
    d_ewr.append(d_e)
    p_l = isAirport(row['pickup_latitude'],row['pickup_longitude'],'LGA')
    p_lga.append(p_l)
    d_l = isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'LGA')
    d_lga.append(d_l)
    if (i+1) % 1000000 == 0:
        print(i+1,datetime.now() - start_time)

df_test['is_pickup_JFK_new'] = p_jfk
df_test['is_dropoff_JFK_new'] = d_jfk
df_test['is_pickup_EWR_new'] = p_ewr
df_test['is_dropoff_EWR_new'] = d_ewr
df_test['is_pickup_LGA_new'] = p_lga
df_test['is_dropoff_LGA_new'] = d_lga
df_test['is_to_from_JFK_new'] = pd_jfk
print(datetime.now())
df_test.info()

2018-09-19 08:07:19.467127
2018-09-19 08:07:20.489010
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 15 columns):
key                   9914 non-null object
pickup_datetime       9914 non-null datetime64[ns]
pickup_longitude      9914 non-null float32
pickup_latitude       9914 non-null float32
dropoff_longitude     9914 non-null float32
dropoff_latitude      9914 non-null float32
passenger_count       9914 non-null uint8
distance_miles        9914 non-null float32
is_pickup_JFK_new     9914 non-null bool
is_dropoff_JFK_new    9914 non-null bool
is_pickup_EWR_new     9914 non-null bool
is_dropoff_EWR_new    9914 non-null bool
is_pickup_LGA_new     9914 non-null bool
is_dropoff_LGA_new    9914 non-null bool
is_to_from_JFK_new    9914 non-null bool
dtypes: bool(7), datetime64[ns](1), float32(5), object(1), uint8(1)
memory usage: 426.1+ KB


In [18]:
df_test['distance_to_center'] = distance(nyc[1], nyc[0], df_test.pickup_latitude, df_test.pickup_longitude)
df_test['hour'] = df_test.pickup_datetime.apply(lambda t: t.hour)
df_test['weekday'] = df_test.pickup_datetime.apply(lambda t: t.weekday())
df_test['year'] = df_test.pickup_datetime.apply(lambda t: t.year)

In [19]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance_miles,is_pickup_JFK_new,is_dropoff_JFK_new,is_pickup_EWR_new,is_dropoff_EWR_new,is_pickup_LGA_new,is_dropoff_LGA_new,is_to_from_JFK_new,distance_to_center,hour,weekday,year
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24,-73.973320,40.763805,-73.981430,40.743835,1,1.366828,False,False,False,False,False,False,False,1.366828,13,1,2015
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24,-73.986862,40.719383,-73.998886,40.739201,1,1.366828,False,False,False,False,False,False,False,4.354117,13,1,2015
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44,-73.982521,40.751259,-73.979652,40.746140,1,0.000000,False,False,False,False,False,False,False,2.192740,11,5,2011
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12,-73.981163,40.767807,-73.990448,40.751637,1,1.366828,False,False,False,False,False,False,False,1.035088,21,5,2012
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12,-73.966049,40.789776,-73.988564,40.744427,1,3.226874,False,False,False,False,False,False,False,0.000000,21,5,2012


Save in feather format to load the dataset quickly into ml models without recomputing all the features.

In [20]:
path_train = '../../datasets/kaggle/new-york-city-taxi-fare-prediction/train2.feather'
feather.write_dataframe(df_train, path_train)

In [21]:
path_test = '../../datasets/kaggle/new-york-city-taxi-fare-prediction/test2.feather'
feather.write_dataframe(df_test, path_test)